In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import tqdm
import firedrake
from firedrake import (
    max_value, Constant, exp, sin, cos, inner, tr, sym, grad, dx, ds, dS, avg, jump
)
import irksome
from irksome import Dt

In [ ]:
mesh = firedrake.UnitDiskMesh(4)
n = firedrake.FacetNormal(mesh)
x = firedrake.SpatialCoordinate(mesh)

In [ ]:
dg1 = firedrake.FiniteElement("DG", "triangle", 1)
cg2 = firedrake.FiniteElement("CG", "triangle", 2)
Q = firedrake.FunctionSpace(mesh, dg1)
S = firedrake.FunctionSpace(mesh, cg2)

In [ ]:
r = Constant(0.25)
b_0 = Constant(1.0)
expr = b_0 * exp(-inner(x, x) / r**2)
b = firedrake.Function(S).interpolate(expr)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(b, axes=ax)
fig.colorbar(colors);

In [ ]:
h_0 = Constant(0.1)
R = Constant(0.125)
expr = h_0 * exp(-inner(x, x) / R**2)
h = firedrake.Function(Q).project(expr)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(h, axes=ax)
fig.colorbar(colors);

In [ ]:
ρ = Constant(1.0)
g = Constant(1.0)
μ = Constant(1.0)
γ = Constant(0.1)

In [ ]:
def ε(v):
    return sym(grad(v))

def momentum_form(**kwargs):
    field_names = ("velocity", "thickness", "surface", "test_function")
    u, h, s, v = map(kwargs.get, field_names)

    parameter_names = ("viscosity", "friction")
    μ, γ = map(kwargs.get, parameter_names)

    F_viscosity = h * μ * (inner(ε(u), ε(v)) + tr(ε(u)) * tr(ε(v))) * dx
    F_friction = γ * inner(u, v) * dx
    F_gravity = (
        -ρ * g * h * inner(grad(s), v) * dx +
        ρ * g * avg(h) * inner(jump(s, n), avg(v)) * dS
    )

    return F_viscosity + F_friction - F_gravity

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, cg2)
u = firedrake.Function(V)
v = firedrake.TestFunction(V)

fields = {"velocity": u, "thickness": h, "surface": b + h}
parameters = {"viscosity": μ, "friction": γ}

F = momentum_form(**fields, **parameters, test_function=v)
bc = firedrake.DirichletBC(V, 0, "on_boundary")
firedrake.solve(F == 0, u, bc)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.streamplot(u, resolution=0.05, axes=ax)
fig.colorbar(colors);

In [ ]:
Z = Q * V
z = firedrake.Function(Z)
z.subfunctions[1].assign(u)
z.subfunctions[0].assign(h)

h, u = firedrake.split(z)
ϕ, v = firedrake.TestFunctions(Z)

In [ ]:
def mass_form(**kwargs):
    field_names = ("thickness", "velocity", "accumulation", "test_function")
    h, u, a, ϕ = map(kwargs.get, field_names)

    F_cells = (Dt(h) * ϕ - inner(h * u, grad(ϕ)) - a * ϕ) * dx
    f = h * max_value(0, inner(u, n))
    F_facets = jump(f) * jump(ϕ) * dS

    return F_cells + F_facets

In [ ]:
a = Constant(0.0)  # TODO: make it do something

In [ ]:
fields = {"velocity": u, "thickness": h, "surface": b + h}
parameters = {"viscosity": μ * exp(-Q / (R * T)), "friction": γ}
F_velocity = momentum_form(**fields, **parameters, test_function=v)
F_mass = mass_form(thickness=h, test_function=ϕ, velocity=u, accumulation=a)
F = F_velocity + F_mass

bcs = firedrake.DirichletBC(Z.sub(1), Constant((0.0, 0.0)), "on_boundary")

final_time = 10.0
num_steps = 1000
dt = Constant(final_time / num_steps)
t = Constant(0.0)

method = irksome.RadauIIA(2)
solver = irksome.TimeStepper(F, method, t, dt, z, bcs=bcs)

In [ ]:
zs = [z.copy(deepcopy=True)]
for step in tqdm.trange(num_steps):
    solver.advance()
    z.subfunctions[0].project(max_value(0, z.subfunctions[0]))
    zs.append(z.copy(deepcopy=True))

In [ ]:
%%capture

fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.axis("off")

h = zs[0].subfunctions[0]
hmin, hmax = h.dat.data_ro.min(), h.dat.data_ro.max()
kw = {"num_sample_points": 1, "shading": "gouraud"}
colors = firedrake.tripcolor(h, vmin=hmin, vmax=hmax, axes=ax, **kw)
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=1)

def animate(z):
    h = z.subfunctions[0]
    colors.set_array(fn_plotter(h))

animation = FuncAnimation(fig, animate, zs, interval=1e3/60)

In [ ]:
HTML(animation.to_html5_video())